# Pfizer Territory Optimization - STEP 1 Complete Analysis

**Project:** Decision Modelling 2025-26  
**Deadline:** December 18, 2024  
**Problem:** Multi-objective optimization for Sales Representative territory assignment  

---

## Problem Overview

- **4 Sales Representatives (SRs)** need to be assigned to **22 bricks (territories)**
- **Two competing objectives:**
  1. Minimize total travel distance
  2. Minimize disruption to current assignments
- **Constraints:**
  - Each brick assigned to exactly one SR
  - Workload balance: each SR gets between [wl_min, wl_max] workload

---

## Table of Contents
1. [Data Loading & Validation](#section1)
2. [Current Assignment Analysis](#section2)
3. [Model 1: Minimize Distance](#section3)
4. [Model 2: Minimize Disruption](#section4)
5. [Pareto Frontier Analysis](#section5)
6. [Scenario Comparison](#section6)
7. [Key Insights & Recommendations](#section7)

---
<a id='section1'></a>
## 1. Data Loading & Validation

In [ ]:
# Import required libraries
from pfizer_optimization import PfizerOptimization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from IPython.display import Image, display

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

In [ ]:
# Initialize optimizer and load data
opt = PfizerOptimization(data_path='data/')
opt.load_data()

In [ ]:
# Validate data
print("\n=" * 70)
print("DATA VALIDATION")
print("=" * 70)

print(f"\nNumber of bricks: {len(opt.bricks)}")
print(f"Number of SRs: {len(opt.srs)}")
print(f"Total workload: {sum(opt.workload.values()):.4f}")
print(f"Target per SR: {sum(opt.workload.values()) / len(opt.srs):.4f}")

print("\nSR Office Locations (Center Bricks):")
for sr, brick in opt.center_bricks.items():
    print(f"  SR{sr}: Brick {brick}")

print("\nWorkload Statistics:")
workload_df = pd.DataFrame(list(opt.workload.items()), columns=['Brick', 'Workload'])
print(workload_df.describe())

---
<a id='section2'></a>
## 2. Current Assignment Analysis

In [ ]:
# Analyze current assignment
current_sol = opt.analyze_current_assignment()
opt.print_solution_comparison(current_sol, "Current Territory Assignment")

In [ ]:
# Identify workload violations
print("\n=" * 70)
print("CURRENT ASSIGNMENT PROBLEMS")
print("=" * 70)

violations = []
for sr, wl in current_sol['workloads'].items():
    if wl < 0.8:
        violations.append(f"SR{sr}: UNDERLOADED ({wl:.4f} < 0.8)")
    elif wl > 1.2:
        violations.append(f"SR{sr}: OVERLOADED ({wl:.4f} > 1.2)")

if violations:
    print("\nWorkload Constraint Violations:")
    for v in violations:
        print(f"  ✗ {v}")
else:
    print("\n✓ All workload constraints satisfied")

print(f"\nTotal Distance: {current_sol['total_distance']:.2f} km")

---
<a id='section3'></a>
## 3. Model 1: Minimize Distance

**Objective:** Minimize total travel distance  
**Constraints:**  
- Each brick assigned to exactly one SR
- Workload balance: 0.8 ≤ workload ≤ 1.2 for each SR

In [ ]:
# Solve Model 1
print("\n" + "="*70)
print("MODEL 1: MINIMIZE DISTANCE")
print("="*70)

m1, sol1 = opt.model_1_minimize_distance(wl_min=0.8, wl_max=1.2, verbose=True)
opt.print_solution_comparison(sol1, "Model 1: Minimum Distance Solution")

In [ ]:
# Calculate improvements
A = opt.create_current_assignment_matrix()
disruption1 = opt._calculate_disruption(sol1, A)
distance_improvement = current_sol['total_distance'] - sol1['total_distance']
distance_pct = (distance_improvement / current_sol['total_distance']) * 100

print("\nModel 1 Performance Metrics:")
print(f"  Total Distance: {sol1['total_distance']:.2f} km")
print(f"  Distance Improvement: {distance_improvement:.2f} km ({distance_pct:.1f}%)")
print(f"  Disruption: {disruption1:.4f}")

# Count reassignments
reassignments = sum(1 for i in opt.bricks for j in opt.srs 
                   if i in sol1['assignment'][j] and A[(i, j)] == 0)
print(f"  Bricks Reassigned: {reassignments}/{len(opt.bricks)}")
print(f"  Retention Rate: {((len(opt.bricks) - reassignments) / len(opt.bricks) * 100):.1f}%")

---
<a id='section4'></a>
## 4. Model 2: Minimize Disruption

**Objective:** Minimize disruption (weighted by workload)  
**Disruption Metric:** Sum of index values for reassigned bricks  
**Constraints:** Same as Model 1

In [ ]:
# Solve Model 2
print("\n" + "="*70)
print("MODEL 2: MINIMIZE DISRUPTION")
print("="*70)

m2, sol2 = opt.model_2_minimize_disruption(wl_min=0.8, wl_max=1.2, verbose=True)
opt.print_solution_comparison(sol2, "Model 2: Minimum Disruption Solution")

In [ ]:
# Calculate metrics
disruption2 = opt._calculate_disruption(sol2, A)
reassignments2 = sum(1 for i in opt.bricks for j in opt.srs 
                    if i in sol2['assignment'][j] and A[(i, j)] == 0)

print("\nModel 2 Performance Metrics:")
print(f"  Total Distance: {sol2['total_distance']:.2f} km")
print(f"  Distance vs Current: {sol2['total_distance'] - current_sol['total_distance']:.2f} km")
print(f"  Disruption: {disruption2:.4f}")
print(f"  Bricks Reassigned: {reassignments2}/{len(opt.bricks)}")
print(f"  Retention Rate: {((len(opt.bricks) - reassignments2) / len(opt.bricks) * 100):.1f}%")

In [ ]:
# Compare Models
comparison_df = pd.DataFrame({
    'Metric': ['Total Distance (km)', 'Disruption', 'Bricks Reassigned', 'Retention Rate (%)'],
    'Current': [current_sol['total_distance'], 0.0, 0, 100.0],
    'Model 1 (Min Dist)': [sol1['total_distance'], disruption1, reassignments, 
                           (len(opt.bricks) - reassignments) / len(opt.bricks) * 100],
    'Model 2 (Min Disr)': [sol2['total_distance'], disruption2, reassignments2,
                           (len(opt.bricks) - reassignments2) / len(opt.bricks) * 100]
})

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))

---
<a id='section5'></a>
## 5. Pareto Frontier Analysis

Using the **epsilon-constraint method**, we generate the Pareto frontier:  
- **Primary objective:** Minimize distance  
- **Constraint:** Disruption ≤ ε (varying)  

We analyze **3 workload scenarios:**
1. **Scenario 1:** [0.8, 1.2] - Wide bounds (flexible)
2. **Scenario 2:** [0.85, 1.15] - Medium bounds
3. **Scenario 3:** [0.9, 1.1] - Tight bounds (strict)

In [ ]:
# Load pre-computed Pareto results
with open('pareto_results.pkl', 'rb') as f:
    pareto_data = pickle.load(f)

pareto_results = pareto_data['pareto_frontiers']
scenarios = pareto_data['scenarios']

print("Loaded Pareto frontier data:")
for scenario_name, df in pareto_results.items():
    print(f"  {scenario_name}: {len(df)} solutions")

In [ ]:
# Display summary table
summary_df = pd.read_csv('pareto_summary.csv')
print("\n" + "="*70)
print("PARETO FRONTIER SUMMARY")
print("="*70)
print(summary_df.to_string(index=False))

In [ ]:
# Display Pareto comparison plot
display(Image('pareto_comparison.png'))

In [ ]:
# Scenario 1 detailed plot
print("\nScenario 1: [0.8, 1.2] - Detailed Analysis")
display(Image('pareto_Scenario_1_0.8_1.2.png'))

In [ ]:
# Show some Pareto-optimal solutions for Scenario 1
s1_df = pareto_results['Scenario 1: [0.8, 1.2]']
print("\nScenario 1: Sample Pareto-Optimal Solutions")
print(s1_df[['distance', 'disruption']].head(10).to_string(index=False))

---
<a id='section6'></a>
## 6. Scenario Comparison

We compare the three workload scenarios to understand the impact of workload flexibility on solution quality.

In [ ]:
# Display workload comparisons
print("\nWorkload Distribution Comparisons:\n")

print("Scenario 1: [0.8, 1.2]")
display(Image('workload_Scenario_1_0.8_1.2.png'))

print("\nScenario 2: [0.85, 1.15]")
display(Image('workload_Scenario_2_0.85_1.15.png'))

print("\nScenario 3: [0.9, 1.1]")
display(Image('workload_Scenario_3_0.9_1.1.png'))

In [ ]:
# Display assignment heatmaps for Scenario 1
print("\nAssignment Heatmaps (Scenario 1):\n")

print("Min Distance Solution:")
display(Image('assignment_heatmap_scenario1_mindist.png'))

print("\nMin Disruption Solution:")
display(Image('assignment_heatmap_scenario1_mindisr.png'))

In [ ]:
# Analyze trade-offs across scenarios
print("\n" + "="*70)
print("SCENARIO TRADE-OFF ANALYSIS")
print("="*70)

for scenario_name, df in pareto_results.items():
    min_dist = df['distance'].min()
    max_dist = df['distance'].max()
    min_disr = df['disruption'].min()
    max_disr = df['disruption'].max()
    
    improvement = ((187.41 - min_dist) / 187.41) * 100
    
    print(f"\n{scenario_name}")
    print(f"  Best Distance: {min_dist:.2f} km ({improvement:.1f}% improvement)")
    print(f"  Worst Distance: {max_dist:.2f} km")
    print(f"  Distance Range: {max_dist - min_dist:.2f} km")
    print(f"  Min Disruption: {min_disr:.4f}")
    print(f"  Max Disruption: {max_disr:.4f}")
    print(f"  Pareto Solutions: {len(df)}")

---
<a id='section7'></a>
## 7. Key Insights & Recommendations

### Key Findings:

#### 1. Current Assignment Issues
- **SR2 is overloaded:** 1.3377 > 1.2 (violation)
- **SR3 is underloaded:** 0.7048 < 0.8 (violation)
- **Total distance:** 187.41 km (not optimal)

#### 2. Optimization Potential
- **Model 1 (Min Distance):** Achieves 11.4% improvement with wide bounds [0.8, 1.2]
- **Model 2 (Min Disruption):** Maintains stability but increases distance slightly
- **Trade-off:** Distance improvements require some reassignments

#### 3. Workload Flexibility Impact
- **Wide bounds [0.8, 1.2]:** Better distance optimization (165.96 km)
- **Tight bounds [0.9, 1.1]:** More balanced workload but higher distance (171.68 km)
- **Difference:** ~6 km penalty for stricter workload balance

#### 4. Pareto Frontier Characteristics
- Multiple Pareto-optimal solutions available for decision-makers
- Clear trade-off curve between distance and disruption
- Scenario 1 offers most flexibility and best solutions

### Recommendations:

1. **Implement optimization:** Current assignment violates workload constraints
2. **Choose scenario based on priorities:**
   - If distance is critical → Use Scenario 1 solution
   - If workload balance is critical → Use Scenario 3 solution
   - If stability matters → Choose Pareto solution with low disruption
3. **Consider Pareto middle ground:** Solutions with moderate distance improvement and low disruption
4. **Next steps:** Implement partial brick assignments (STEP 2) for even better solutions

In [ ]:
# Recommend a balanced solution from Scenario 1
s1_df = pareto_results['Scenario 1: [0.8, 1.2]']

# Find solution with good distance but low disruption
# Normalize both objectives and find compromise
s1_df['norm_dist'] = (s1_df['distance'] - s1_df['distance'].min()) / (s1_df['distance'].max() - s1_df['distance'].min())
s1_df['norm_disr'] = (s1_df['disruption'] - s1_df['disruption'].min()) / (s1_df['disruption'].max() - s1_df['disruption'].min())
s1_df['compromise'] = s1_df['norm_dist'] + s1_df['norm_disr']

best_compromise_idx = s1_df['compromise'].idxmin()

print("\n" + "="*70)
print("RECOMMENDED SOLUTION (Balanced Compromise)")
print("="*70)
print(f"\nScenario: [0.8, 1.2]")
print(f"Distance: {s1_df.loc[best_compromise_idx, 'distance']:.2f} km")
print(f"Disruption: {s1_df.loc[best_compromise_idx, 'disruption']:.4f}")
print(f"Improvement vs Current: {187.41 - s1_df.loc[best_compromise_idx, 'distance']:.2f} km")
print(f"\nThis solution offers a good balance between distance minimization and")
print(f"maintaining stability (low disruption to current assignments).")

---

## Conclusion

**STEP 1 Complete:** We have successfully:
- ✅ Implemented Model 1 (Minimize Distance)
- ✅ Implemented Model 2 (Minimize Disruption)
- ✅ Generated Pareto frontiers using epsilon-constraint method
- ✅ Analyzed 3 workload scenarios: [0.8, 1.2], [0.85, 1.15], [0.9, 1.1]
- ✅ Created visualizations and comprehensive analysis

**Next Steps (STEP 2 & 3):**
- Implement partial brick assignments (allow splitting bricks)
- Optimize new SR office locations
- Add center brick optimization (variable office locations)
- Generate final report and presentation

---

**Files Generated:**
- `pfizer_optimization.py` - Main optimization module
- `pareto_analysis.py` - Complete Pareto analysis script
- `pareto_results.pkl` - Pickled results for reuse
- `pareto_*.csv` - Pareto frontier data
- `pareto_*.png` - Pareto plots
- `workload_*.png` - Workload comparisons
- `assignment_heatmap_*.png` - Assignment visualizations

---